In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
import coffea.util as coffea_util
import awkward as ak
from dask.distributed import Client
#plotting
import matplotlib.pyplot as plt
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from analysis.tools import ffschema, sidm_processor
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)

<module 'analysis.tools.sidm_processor' from '/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/sidm_processor.py'>

In [8]:
fileset = {
    "2Mu2E_100GeV_1p2GeV_9p6mm": [
        "root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_1.root",
    ],
}

runner = processor.Runner(
    #executor=processor.DaskExecutor(client=Client("tls://localhost:8786")),
    executor=processor.IterativeExecutor(),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "4mu",
        "2mu2e",
    ],
    ["base"]
)

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)

Output()

In [9]:
output

{'out': {'2Mu2E_100GeV_1p2GeV_9p6mm': {'cutflow': {'4mu': <analysis.tools.cutflow.Cutflow at 0x7f1020c186a0>,
    '2mu2e': <analysis.tools.cutflow.Cutflow at 0x7f10104bdcd0>},
   'hists': {'pv_n': Hist(
      StrCategory(['4mu', '2mu2e'], name='channel', label='channel'),
      Regular(50, 0, 100, name='pv_n', label='pv_n'),
      storage=Weight()) # Sum: WeightedSum(value=1431.94, variance=2029.17),
    'pv_ndof': Hist(
      StrCategory(['4mu', '2mu2e'], name='channel', label='channel'),
      Regular(25, 0, 100, name='pv_ndof', label='pv_ndof'),
      storage=Weight()) # Sum: WeightedSum(value=35537.6, variance=52587.1) (WeightedSum(value=38312.7, variance=56727.3) with flow),
    'pv_z': Hist(
      StrCategory(['4mu', '2mu2e'], name='channel', label='channel'),
      Regular(100, -50, 50, name='pv_z', label='pv_z'),
      storage=Weight()) # Sum: WeightedSum(value=38312.7, variance=56727.3),
    'pv_rho': Hist(
      StrCategory(['4mu', '2mu2e'], name='channel', label='channel'),
